# Logging in Python

Knowing what your python program is doing is essential to understanding if it is working correctly. This seems like an obvious statement, but a lot of the time we do this sort of verifications without thinking about it. You could be using:
- Print statements
- A debugger & breakpoints
- Output verification (data files or graphs)
- Other, less quantitative methods:
    - Is this program running too long?
    - Is this program using any cpu?
    - etc.

Unfortunately, there are a few problems with these methods when we come to larger tasks, such as big monte-carlo simulation:

- Print statements are very slow and you might not want to print everything all of the time
- Debuggers are great for development, but you may not be at your computer when something fails
- Running a program in a debugger is slower than running it without a debugger
- You don't want to constantly be checking your outputs for x hours straight.
- You don't want to constantly be checking your cpu usage.
- "Too long" might not have much meaning, depending on the algorithm you are performing.

Wouldn't it be nice if your program produced a report as it went in which it recorded things you wanted to know and any errors it found?

**THIS** is where logging comes in.

## The basics of logging


Logging is much like a much more highly controlled print system. Anywhere you might put a `print()` you can put a log statement instead.

In Python, it is split into a few classes that work with each other to customise the log output:

- Loggers expose the interface that application code directly uses.
- Handlers send the log records (created by loggers) to the appropriate destination (e.g. the screen or a file)
- Filters provide a finer grained facility for determining which log records to output.
- Formatters specify the layout of log records in the final output.


## Logging levels

Logging in python has a system to organise and filter log statements by **level** (aka priority). In order from least severe to most severe these are as follows:
- `DEBUG`
- `INFO`
- `WARNING`
- `ERROR`
- `CRITICAL`

There are no absolute rules for how to use these, but the following sections give a good framework within which to work.

---

### `DEBUG`
Use debug everywhere as a fine-scale feedback system for large jobs. For instance if you were looping through a file row by row, you might add a debug log entry saying `Working on row x`, and maybe even a debug entry for every operation you are doing.

### `INFO`
The `INFO` level is your general purpose logging level. It can be used for coarse status updates and generally tracking the control flow as the program runs. Log entries at this level might look like the following:
- `Loading data...`
- `Fitting curves (10/100)`
- `Rendering graphs`

### `WARNING`
This log level is used when something is out of the ordinary, but in a manner which is handled or known about and mitigated. For example:
- `Rate parameter not supplied, falling back to default (0.5)`
- `Data contains 0s, skipping log transformation...`
- `Not enough points to fit a 5 parameter curve (3 points in sample), skipping...`

### `ERROR`
Errors occur when something big went wrong in a non-fatal way. This can be a bit hard to define, but safe to say if you're really annoyed that it's occuring, it's probably an `ERROR`. Examples include:
- `File 34/60 unreadable or missing`
- `Data file empty`
- `Unexpected error in fitting curve 2/5 for Species 18`
- `Data has 0 variance and is thus identical`

### `CRITICAL`
You do not want to see `CRITICAL` errors. These are a sign that something vital has gone wrong which makes the program inviable to execute past that point. It could also be a sign that something has gone wrong which may have had lasting side-effects that require cleaning up or mitigating. Examples could be:
- `Cannot access database`
- `No cores detected for multithreading`
- `Missing system libraries for GDAL`
- `Error when performing in-place edit to file. File may now be corrupt!`

---

To put these in a slightly more intuitive way, here are my translations of log levels in terms of how they should make you feel when you see them:
- `DEBUG` - "Cool, literally everything works."
- `INFO` - "Nice to know how things are progressing."
- `WARNING` - "Hmm."
- `ERROR` - "Bugger."
- `CRITICAL` - "I'm going to set this computer on fire in a minute..."

## Implementing logging

Let's implement logging in a script. Currently it is in a state which produces a lot of useless output. Take a minute to have a look and work out how the script works.

In [1]:
# Some setup
import random

class SithError(Exception):
    pass


# Main script

def train_padawans(n):
    print(f"Training {n} padawan/s")
    trained_successfully = 0
    for i in range(n):
        try:
            print(f"Training padawan {i}")
            train(i)
            trained_successfully += 1
        except SithError:
            print("Padawan has fallen to the dark side.")

def train(padawan):
    lightpoints = random.randint(5,10)
    darkpoints = random.randint(0,7)
    print(f"Light = {lightpoints}, Dark = {darkpoints}")
    if darkpoints > lightpoints:
        raise SithError
    else:
        return 0
    
train_padawans(20)

Training 20 padawan/s
Training padawan 0
Light = 8, Dark = 3
Training padawan 1
Light = 7, Dark = 6
Training padawan 2
Light = 5, Dark = 3
Training padawan 3
Light = 5, Dark = 2
Training padawan 4
Light = 10, Dark = 2
Training padawan 5
Light = 6, Dark = 6
Training padawan 6
Light = 6, Dark = 7
Padawan has fallen to the dark side.
Training padawan 7
Light = 10, Dark = 7
Training padawan 8
Light = 9, Dark = 2
Training padawan 9
Light = 8, Dark = 6
Training padawan 10
Light = 9, Dark = 2
Training padawan 11
Light = 7, Dark = 2
Training padawan 12
Light = 7, Dark = 6
Training padawan 13
Light = 5, Dark = 2
Training padawan 14
Light = 5, Dark = 2
Training padawan 15
Light = 7, Dark = 2
Training padawan 16
Light = 7, Dark = 0
Training padawan 17
Light = 6, Dark = 4
Training padawan 18
Light = 5, Dark = 1
Training padawan 19
Light = 7, Dark = 1


So let's see if we can make this more informative and reasonable to debug and manage.

## Setting up the logging system

You usually want to set up logging at the start of your script, before doing much else. Only your header and imports usually come beforehand (as anything that is done before setting up logging **can't be logged**)

First let's import logging and get the `root` logger, this is always intantiated and in the absence of a custom logger, this will do the trick for many applications.

In [2]:
import logging

logger = logging.getLogger()

Alternatively if we want a logger specific to our script we could initialise one as follows:

In [3]:
logger = logging.getLogger("MyScript")

Now we have done that we can set up the logging format for the messages. These can be formatted like any string, but for now let's get the log messages in the format `2020-09-08 19:16:16,855 - INFO - Log message`

In [4]:
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

This format string might seem a little arcane, but it relates to the [LogRecord attributes](https://docs.python.org/3/library/logging.html#logrecord-attributes), the data contained in the Log record.

Anyway a formatter on its own will do little good, we need to also add a stream handler to pass data to the console. The handler decides where the log records generated by logging calls are directed.

In [5]:
chandler = logging.StreamHandler()

Good, however this handler is unconfigured and could cause us problems if we use it as-is. Let's set the handler to only transmit log entries at an `INFO` level and higher. We will also set the handler to use the formatter we created above

In [6]:
chandler.setLevel(logging.INFO)
chandler.setFormatter(formatter)

Finally we need to add this handler to the logger we retrieved at the top.

In [7]:
logger.addHandler(chandler)

*Note: the root logger is set by default to only deal with logs at an `ERROR` level or above. In this case we want it to accept and handle log entries at a `DEBUG` level and above.*

In [8]:
logger.setLevel(logging.DEBUG)

And we are all set!

## Logging messages
Now let's test it using log messages at all levels. To do this we need to call `logger.x()` where `x` is the level we want to log at. So to log a `WARNING` you would use `logger.warning()`.

In [9]:
logger.debug("DEBUG message")
logger.info("INFO message")
logger.warning("WARNING message")
logger.error("ERROR message")
logger.critical("CRITICAL message")

2022-10-02 15:08:08,988 - INFO - INFO message
2022-10-02 15:08:08,988 - WARNING - WARNING message
2022-10-02 15:08:08,989 - ERROR - ERROR message
2022-10-02 15:08:08,989 - CRITICAL - CRITICAL message


Notice how we only got 4 messages from the logger even though we wrote 5 commands.

As the `chandler` handler is set to a level of `INFO` it steadfastly ignores any message that it sees that are below that level, thus the `DEBUG` message was not logged. Let's change that for now and see what happens.

In [10]:
chandler.setLevel(logging.DEBUG)


logger.debug("DEBUG message")
logger.info("INFO message")
logger.warning("WARNING message")
logger.error("ERROR message")
logger.critical("CRITICAL message")

2022-10-02 15:08:11,254 - DEBUG - DEBUG message
2022-10-02 15:08:11,255 - INFO - INFO message
2022-10-02 15:08:11,255 - WARNING - WARNING message
2022-10-02 15:08:11,255 - ERROR - ERROR message
2022-10-02 15:08:11,256 - CRITICAL - CRITICAL message


Now we got all the log messages as expected. It is generally good practice to make sure that the logs that you are seeing by default are at `INFO` level and above, and that you're not logging every little thing at `INFO` level either. If a log at `INFO` level is absolutely packed full of things you rarely care about, it's worthwhile reconsidering the levels at which you are logging some of your messages.

With that in mind, let's move the `chandler` back to INFO level.

In [11]:
chandler.setLevel(logging.INFO)

## Inserting Variables

Debug strings are just strings, as such they _can_ be formatted using f-strings as we have done during the course so far.

In [12]:
x = 42
logger.info(f"{x = }")

2022-10-02 15:08:12,996 - INFO - x = 42


However this is actually a bit suboptimal in some cases [see here for more in-depth details](https://blog.pilosus.org/posts/2020/01/24/python-f-strings-in-logging/).

It is therefore best when inputting formatted strings in logging to use the old-style % formatting, and to pass the data to be formatted as an argument to the logger method:

In [13]:
logger.info("x = %d", x)

2022-10-02 15:08:13,537 - INFO - x = 42


This little optimisation makes more difference the more logging calls you make (even ones below the current log level of the handler or logger).

Thus depending on how much you are logging, this little tip could be either a big deal, or a major inconvenience for a minor gain in speed.

## Challenge 1

Try to modify the jedi training script above to use appropriate logging. Note there is no right or wrong answer for this, but try to apply the guidelines at the top of the workbook.

## Logging exceptions
Exceptions can be handled using just a simple `logger.error()` command with an appropriate message, however this will only display the message that you set. An exception usually gives a lot more information, such as a stack trace, and this would be useful to have in the log!

Here is where a special logger method caller `logger.exception()` comes in.

For example:

In [14]:
def div_by_nsub1(n):
    return n/(n-1)

try:
    for i in range(-5, 5):
        print(div_by_nsub1(i))
except Exception:
    logger.error("An error occurred")

2022-10-02 15:08:15,391 - ERROR - An error occurred


0.8333333333333334
0.8
0.75
0.6666666666666666
0.5
-0.0


And now the same thing but using `logger.exception()` instead of `logger.error()`.

In [15]:
def div_by_nsub1(n):
    return n/(n-1)

try:
    for i in range(-5, 5):
        print(div_by_nsub1(i))
except Exception:
    logger.exception("An error occurred")

2022-10-02 15:08:15,939 - ERROR - An error occurred
Traceback (most recent call last):
  File "/tmp/ipykernel_16810/2065718702.py", line 6, in <module>
    print(div_by_nsub1(i))
  File "/tmp/ipykernel_16810/2065718702.py", line 2, in div_by_nsub1
    return n/(n-1)
ZeroDivisionError: division by zero


0.8333333333333334
0.8
0.75
0.6666666666666666
0.5
-0.0


Note the large amount of extra information present in the stack trace alongside the error message itself! Much easier to see exactly where in the code the error was triggered (`line 2, in div_by_nsub1`).

Also worthwhile noting that `logger.exception()` logs at the `ERROR` level.

You don't *need* to use the `logger.exception()` method to write to the log whenever you catch an exception, and in many situations it will just spam up your log with useless information (precisely what we are trying to avoid). If you think that the information contained in the stack trace will be useful then use `logger.exception()`, otherwise another form will do just as well.

## HIGHLY RECOMMENDED READING
At this point, go to the following article, read it in full (should take less than 5 minutes), then bookmark it for future reference.

It provides some really nice ways to log python exceptions and I thoroughly recommend everyone read this before implementing logging in their own code.

---

> [**Exceptional logging of exceptions in Python**](https://www.loggly.com/blog/exceptional-logging-of-exceptions-in-python/)

---

## Logging to a file

The console isn't the only place you might want to put a log. Arguably more useful is a log file that you can investigate and search at your leisure. To create a logger that writes to the file `my.log` in the current directory, you can set up and attach another handler as follows:

In [13]:
fhandler = logging.FileHandler('my.log')
fhandler.setLevel(logging.DEBUG)
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)

Note that it is possible to have different handlers with differing log levels or formatters.

Now let's log some stuff!

In [14]:
logger.debug("DEBUG message")
logger.info("INFO message")
logger.warning("WARNING message")
logger.error("ERROR message")
try:
    x = 1/0
except Exception:
    logger.exception("EXCEPTION + stacktrace")

logger.critical("CRITICAL message")

2020-09-08 21:02:02,469 - INFO - INFO message
2020-09-08 21:02:02,470 - WARNING - WARNING message
2020-09-08 21:02:02,470 - ERROR - ERROR message
2020-09-08 21:02:02,471 - ERROR - EXCEPTION + stacktrace
Traceback (most recent call last):
  File "<ipython-input-14-e48625660833>", line 6, in <module>
    x = 1/0
ZeroDivisionError: division by zero
2020-09-08 21:02:02,471 - CRITICAL - CRITICAL message


And let's look at what is in the log file

In [15]:
with open("my.log", "r") as f:
    for line in f:
        print(line.strip())

2020-09-08 21:02:02,469 - DEBUG - DEBUG message
2020-09-08 21:02:02,469 - INFO - INFO message
2020-09-08 21:02:02,470 - WARNING - WARNING message
2020-09-08 21:02:02,470 - ERROR - ERROR message
2020-09-08 21:02:02,471 - ERROR - EXCEPTION + stacktrace
Traceback (most recent call last):
File "<ipython-input-14-e48625660833>", line 6, in <module>
x = 1/0
ZeroDivisionError: division by zero
2020-09-08 21:02:02,471 - CRITICAL - CRITICAL message


As we can see here, even the bits that are logged at `DEBUG` level are appearing in the log file, but they do NOT appear at the console level.

## Final notes

With these skills you should now be able to set up logging for large scripts easily and use it effectively to provide insight into the inner workings of a script as it is running. A few final tips to see you off:

- You can follow a log file in real time as a program is running using the bash command `tail -f my.log`. You can quit this view using Ctrl+C.
- Implementing a debug mode can be quite a good idea at the top of a file. With a simple boolean value, you can switch between different logging setups, leading to a way to easily turn on a much more detailed view of what is going on.
- There is a package called `logzero` that makes a lot of this stuff even easier, go check it out!
- Logging in R is possible using the `futile.logger` package or a few others, however in my experience it is less powerful. Nonetheless the same reasons for using it apply.